# CO-Analyse (Jahresverlauf, Nord-Süd-Gefälle, Städte)

In [ ]:
# Plot changes throughout the year. Example: Analyse CO concentration per month

# calculate mean CO per month
monthly_co = df.groupby("Month")["Co"].mean()
# monthly_co = df.groupby("Month")["Co"].mean()

# create plot
plt.figure(figsize=(10, 5))
plt.plot(monthly_co.index, monthly_co.values, marker='o', linestyle='-')
plt.xlabel("Monat")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Durchschnittliche CO-Konzentration pro Monat")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.grid(True);

# DaAn diesem Schaubild kann man zeigen, dass es Quatsch ist, alle Städte weltweit über einen Kamm zu scheren;
# Es sagt aber an sich nichts Sinnvolles über die Daten aus.

In [ ]:
# Sinnvoller wäre es, die Daten in Nord- und Südhalbkugel zu trennen und getrennt zu betrachten. Aber das ignoriert immer noch viele relevante Faktoren

# Trennung der Daten in Nord- und Südhalbkugel

# Definieren, welche Länder zur Nord- und Südhalbkugel gehören
northern_hemisphere_countries = {
    "US", "CA", "MX", "DE", "FR", "GB", "RU", "CN", "JP", "IN", "IT", "ES", "PL", "TR", "IR", "KR", "UA", "NL", "BE",
    "CH", "SE", "AT", "NO", "FI", "DK", "GR", "CZ", "HU", "RO", "BG", "PT", "IE", "SK", "HR", "LT", "SI", "LV", "EE"
}
southern_hemisphere_countries = {
    "AU", "NZ", "AR", "BR", "ZA", "CL", "ID", "PE", "BO", "EC", "PY", "UY", "MG"
}

# Daten für Nord- und Südhalbkugel filtern
df_north = df[df["Country"].isin(northern_hemisphere_countries)]
df_south = df[df["Country"].isin(southern_hemisphere_countries)]

# Mittlere CO-Konzentration pro Monat berechnen
monthly_co_north = df_north.groupby("Month")["Co"].mean()
monthly_co_south = df_south.groupby("Month")["Co"].mean()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_co_north.index, monthly_co_north.values, marker='o', linestyle='-', label="Nordhalbkugel", color='b')
plt.plot(monthly_co_south.index, monthly_co_south.values, marker='o', linestyle='-', label="Südhalbkugel", color='r')

plt.xlabel("Monat")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Vergleich der CO-Konzentration auf Nord- und Südhalbkugel")
plt.xticks(range(1, 13))  # Monatsskala 1-12
plt.legend()
plt.grid(True);

In [ ]:
# CO-Werte über das Jahr inklusive Streuung

plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x="Month", y="Co", errorbar="sd", marker="o")
plt.xlabel("Monat")
plt.ylabel("CO-Wert")
plt.title("Kohlenmonoxidwerte Im Jahresverlauf")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.show()

# Die Grafik ist nicht intuitiv aussagekräftig. Die Streuung ist zu stark.

In [ ]:
# Streuung mit Achsenbegrenzung

# Gruppiere nach Monat
grouped = df.groupby("Month")["Co"]
mean = grouped.mean()
std = grouped.std()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(mean.index, mean.values, marker='o', linestyle='-', label='Mittelwert CO')
plt.fill_between(mean.index, mean - std, mean + std, color='blue', alpha=0.2, label='±1 SD')

# Achsen, Titel, Beschriftungen
plt.xlabel("Monat")
plt.ylabel("CO-Konzentration (ppm)")
plt.title("Durchschnittliche CO-Konzentration mit Streuung")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])

# Fokus auf relevanten y-Bereich
plt.ylim(3, 7)  # oder anpassen je nach Datensatz

plt.grid(True)
plt.legend()
plt.show()

# Ergebnis: Die Streuung bedeckt die gesamte Diagrammfläche. Es sollten als die Städte, die die größte Verzerrung bewirken, aussortiert werden.


In [ ]:
co_by_city = df.groupby("City")["Co"].mean().sort_values(ascending=False)
print(co_by_city.head(10))

# Ashkelon ist ein unrealistischer Ausreißer. Siehe separates NB "Ashkelon": Es gibt nur Messwerte für Jan und Feb 2022, und diese schwanken extrem. Vielleicht kaputtes Messgerät.

In [ ]:
# calculate std per city

co_std_by_city = df.groupby("City")["Co"].std().sort_values(ascending=False)
print(co_std_by_city.head(10))

In [ ]:
# combine mean and std to determine cities that cause the bigges distortions.

co_stats = df.groupby("City")["Co"].agg(["mean", "std"]).sort_values(by="mean", ascending=False)
print(co_stats.head(10))

In [ ]:
# The three Israeli cities of Ashkelon, Ashdod and Netanya show highly volatile CO values, measured only in certain months.
# This distorts the graph to an extent that calls for dropping these three cities from further analyes of CO.

In [ ]:
# Drop distorting cities from fürther analyses
exclude_cities = ["Ashkelon", "Ashdod", "Netanya"]
df_cleaned = df[~df["City"].isin(exclude_cities)]

In [ ]:
# Streuung mit Achsenbegrenzung, ohne Ausreißerstädte in Israel

# Gruppiere nach Monat
grouped = df_cleaned.groupby("Month")["Co"]
mean = grouped.mean()
std = grouped.std()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(mean.index, mean.values, marker='o', linestyle='-', label='Mittelwert CO')
plt.fill_between(mean.index, mean - std, mean + std, color='blue', alpha=0.2, label='±1 SD')

# Achsen, Titel, Beschriftungen
plt.xlabel("Monat")
plt.ylabel("CO-Konzentration (ppm)")
plt.title("Durchschnittliche CO-Konzentration mit Streuung")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])

# Fokus auf relevanten y-Bereich
plt.ylim(3, 7)  # oder anpassen je nach Datensatz

plt.grid(True)
plt.legend()
plt.show()

# Ergebnis: Die Streuung bedeckt die gesamte Diagrammfläche. Es sollten als die Städte, die die größte Verzerrung bewirken, aussortiert werden.


In [ ]:
# CO-Werte über das Jahr inklusive Streuung

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_cleaned, x="Month", y="Co", errorbar="sd", marker="o")
plt.xlabel("Monat")
plt.ylabel("CO-Wert")
plt.title("Kohlenmonoxidwerte Im Jahresverlauf")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.show()

# Die Grafik ist immer noch nicht nicht intuitiv aussagekräftig. Die Streuung ist weiterhin zu stark.

In [ ]:
# Ursachenforschung für die starke CO-Streuung

co_std_cleaned = df_cleaned.groupby("City")["Co"].std().sort_values(ascending=False)
print(co_std_cleaned.head(10))

In [ ]:
co_stats_cleaned = df_cleaned.groupby("City")["Co"].agg(["mean", "std"]).sort_values(by="std", ascending=False)
print(co_stats_cleaned.head(10))

In [ ]:
# Weitere Städte mit extremer Varianz entfernen, um eine übersichtliche globale Darstellung zu erzielen

more_extreme = ["Portland", "Mérida", "Zamboanga", "Butuan", "Hạ long", "Oaxaca", "Isfahan", "San luis potosí", "Tabriz", "Tallahassee"]
exclude_cities = ["Ashkelon", "Ashdod", "Netanya"] + more_extreme
df_cleaned2 = df[~df["City"].isin(exclude_cities)]

In [ ]:
# CO-Werte über das Jahr inklusive Streuung, weiter reduziert

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_cleaned2, x="Month", y="Co", marker="o")
plt.fill_between(mean.index, mean - std * 0.5, mean + std * 0.5, alpha=0.2, label='±0.5 SD')
plt.xlabel("Monat")
plt.ylabel("CO-Wert")
plt.title("Kohlenmonoxidwerte Im Jahresverlauf")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.show()

# Die Grafik ist immer noch nicht nicht intuitiv aussagekräftig. Die Streuung ist weiterhin zu stark.

In [ ]:
# Nord- und Südhalbkugel anhand des Features "Latitude" trennen

df_cleaned2.loc[:, "Hemisphere"] = df_cleaned2.loc[:, "Latitude"].apply(lambda x: "Nordhalbkugel" if x >= 0 else "Südhalbkugel")

In [ ]:
df_north = df_cleaned2[df_cleaned2["Hemisphere"] == "Nordhalbkugel"]
df_south = df_cleaned2[df_cleaned2["Hemisphere"] == "Südhalbkugel"]

In [ ]:
# Nordhalbkugel
grouped_north = df_north.groupby("Month")["Co"]
mean_north = grouped_north.mean()
std_north = grouped_north.std()

# Südhalbkugel
grouped_south = df_south.groupby("Month")["Co"]
mean_south = grouped_south.mean()
std_south = grouped_south.std()

# Plot
plt.figure(figsize=(10, 5))

plt.plot(mean_north.index, mean_north, marker='o', label="Nordhalbkugel")
plt.fill_between(mean_north.index, mean_north - std_north, mean_north + std_north, alpha=0.2)

plt.plot(mean_south.index, mean_south, marker='s', label="Südhalbkugel")
plt.fill_between(mean_south.index, mean_south - std_south, mean_south + std_south, alpha=0.2)

plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.xlabel("Monat")
plt.ylabel("CO-Konzentration (ppm)")
plt.title("Saisonale CO-Muster nach Hemisphäre")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Mittlere CO-Konzentration pro Monat berechnen
monthly_co_north = df_north.groupby("Month")["Co"].mean()
monthly_co_south = df_south.groupby("Month")["Co"].mean()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_co_north.index, monthly_co_north.values, marker='o', linestyle='-', label="Nordhalbkugel", color='b')
plt.plot(monthly_co_south.index, monthly_co_south.values, marker='o', linestyle='-', label="Südhalbkugel", color='r')

plt.xlabel("Monat")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Vergleich der CO-Konzentration auf Nord- und Südhalbkugel")
plt.xticks(range(1, 13))  # Monatsskala 1-12
plt.legend()
plt.grid(True);

In [ ]:
# Mittlere CO-Konzentration pro Monat berechnen- Median statt Mean, weil robuster gegen Ausreißer
monthly_co_north = df_north.groupby("Month")["Co"].median()
monthly_co_south = df_south.groupby("Month")["Co"].median()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_co_north.index, monthly_co_north.values, marker='o', linestyle='-', label="Nordhalbkugel", color='b')
plt.plot(monthly_co_south.index, monthly_co_south.values, marker='o', linestyle='-', label="Südhalbkugel", color='r')

plt.xlabel("Monat")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Vergleich der CO-Konzentration auf Nord- und Südhalbkugel")
plt.xticks(range(1, 13))  # Monatsskala 1-12
plt.legend()
plt.grid(True);

In [ ]:
# sns.histplot(df_north["Co"], kde=True)
# sns.histplot(df_south["Co"], kde=True)

plt.figure(figsize=(10, 5))

# Nordhalbkugel
sns.histplot(df_north["Co"], kde=True, label="Nordhalbkugel", stat="density", element="step", fill=True)

# Südhalbkugel
sns.histplot(df_south["Co"], kde=True, label="Südhalbkugel", stat="density", element="step", fill=True)

# Achsen & Legende
plt.xlabel("CO-Konzentration (ppm)")
plt.ylabel("Dichte")
plt.title("Verteilung der CO-Werte nach Hemisphäre")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

# Histogramm Nordhalbkugel
sns.histplot(df_north["Co"], kde=True, stat="density", label="Nordhalbkugel", color="steelblue", fill=True)

# Histogramm Südhalbkugel
sns.histplot(df_south["Co"], kde=True, stat="density", label="Südhalbkugel", color="darkorange", fill=True)

# Zoom auf interessanten Bereich
plt.xlim(0, 20)    # X-Achse (CO-Werte) begrenzen
plt.ylim(0, 0.7)   # Y-Achse (Dichte) begrenzen

# Titel & Achsen
plt.xlabel("CO-Konzentration (ppm)")
plt.ylabel("Dichte")
plt.title("CO-Verteilung nach Hemisphäre (vergrößerter Bereich)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Mittlere CO-Konzentration pro Land berechnen
country_co_avg = df.groupby("Country")["Co"].mean().sort_values(ascending=False)

# Barplot erstellen
plt.figure(figsize=(12, 6))
country_co_avg.plot(kind='bar', color='b', alpha=0.7)
plt.xlabel("Land")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Durchschnittliche CO-Konzentration pro Land")
plt.xticks(rotation=90)  # Länderbeschriftung drehen für bessere Lesbarkeit
plt.grid(axis='y', linestyle='--', alpha=0.7);

In [ ]:
# Mittlere CO-Konzentration pro Land berechnen

# Mindestanzahl an CO-Messwerten pro Land, um in die Analyse aufgenommen zu werden
min_measurements = 100  # Falls nötig, anpassen

# Anzahl der CO-Messwerte pro Land berechnen
country_co_counts = df.groupby("Country")["Co"].count()

# Nur Länder behalten, die mindestens `min_measurements` Messwerte haben
valid_countries = country_co_counts[country_co_counts >= min_measurements].index

# Gefilterten DataFrame mit diesen Ländern erstellen
df_valid_countries = df[df["Country"].isin(valid_countries)]

# Mittlere CO-Konzentration für diese Länder berechnen
country_co_avg_filtered = df_valid_countries.groupby("Country")["Co"].mean().sort_values(ascending=False)

# Falls nach der Filterung noch Daten vorhanden sind, Plot erstellen
if not country_co_avg_filtered.empty:
    plt.figure(figsize=(12, 6))
    country_co_avg_filtered.plot(kind='bar', color='b', alpha=0.7)
    plt.xlabel("Land")
    plt.ylabel("Mittlere CO-Konzentration")
    plt.title("Durchschnittliche CO-Konzentration pro Land (nur Länder mit ausreichenden Messwerten)")
    plt.xticks(rotation=90)  # Länderbeschriftung drehen für bessere Lesbarkeit
    plt.grid(axis='y', linestyle='--', alpha=0.7)
else:
    print("Keine ausreichenden Daten für CO-Werte in den Ländern verfügbar.")


In [ ]:
country_co_avg_filtered.head(20).plot(kind='bar')

In [ ]:
# Vergleich von Schadstoffen in verschiedenen Städten

# Liste der relevanten Schadstoff-Spalten (falls sie in den Daten vorhanden sind)
pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]

# DataFrame mit nur den relevanten Spalten (fehlende Werte entfernen)
df_pollutants = df[pollutants].dropna()

# Korrelationsmatrix berechnen
corr_matrix = df_pollutants.corr()

# Heatmap der Korrelationen erstellen
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Korrelation zwischen CO und anderen Schadstoffen");

In [ ]:
df[["Co", "No2"]].corr()

In [ ]:
# Ab hier Variablen korrigieren!!!

# # Korrelation Schadstoffe und Wettervariablen

# # Liste der Schadstoffe und Wettervariablen
# pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]
# weather_vars = ["temperature", "pressure", "humidity", "dew", "wind-speed", "wind-gust"]

# # DataFrame mit nur den relevanten Spalten (fehlende Werte entfernen)
# df_pollutants_weather = df[pollutants + weather_vars].dropna()

# # Korrelationsmatrix berechnen
# corr_matrix_weather = df_pollutants_weather.corr()

# # Heatmap der Korrelationen zwischen Schadstoffen & Wettervariablen
# plt.figure(figsize=(10, 6))
# sns.heatmap(corr_matrix_weather, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
# plt.title("Korrelation zwischen Schadstoffen und Wetterfaktoren");

In [ ]:
df[["co", "temperature"]].corr()

In [ ]:
# Schadstoffbelastung über die Zeit in verschiedenen Städten

# Durchschnittliche Schadstoffwerte pro Jahr berechnen
pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]
yearly_trends = df.groupby("Year")[pollutants].mean()

# Liniendiagramm für langfristige Trends erstellen
plt.figure(figsize=(12, 6))
for pollutant in pollutants:
    plt.plot(yearly_trends.index, yearly_trends[pollutant], marker='o', linestyle='-', label=pollutant)

plt.xlabel("Jahr")
plt.ylabel("Mittlere Konzentration")
plt.title("Langfristige Entwicklung der Schadstoffwerte")
plt.legend()
plt.grid(True);

In [ ]:
# Ohne 2014 und 2025, weil zu wenige Daten

# Schadstoffe, die analysiert werden sollen
pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]

# Falls "year" als String gespeichert ist, in numerischen Wert umwandeln
df["year"] = pd.to_numeric(df["Year"], errors="coerce")

# Durchschnittliche Schadstoffwerte pro Jahr berechnen, aber 2014 & 2025 ausschließen
yearly_trends = df.groupby("Year")[pollutants].mean()
yearly_trends = yearly_trends.loc[(yearly_trends.index > 2014) & (yearly_trends.index < 2025)]

# Liniendiagramm für langfristige Trends erstellen
plt.figure(figsize=(12, 6))
for pollutant in pollutants:
    plt.plot(yearly_trends.index, yearly_trends[pollutant], marker='o', linestyle='-', label=pollutant)

plt.xlabel("Jahr")
plt.ylabel("Mittlere Konzentration")
plt.title("Langfristige Entwicklung der Schadstoffwerte (ohne 2014 & 2025)")
plt.legend()
plt.grid(True);

In [ ]:
num_cities_with_data = df.loc[:, ["City", "Co", "No2", "So2", "O3", "Pm25", "Pm10"]].dropna(subset=["Co", "No2", "So2", "O3", "Pm25", "Pm10"], how="all")["City"].nunique()
print(f"Anzahl der Städte mit mindestens einem Messwert: {num_cities_with_data}")

In [ ]:
    import matplotlib.pyplot as plt

    # Schadstoffe, die analysiert werden sollen
    pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]

    # Länder nach Regionen gruppieren
    regions = {
        "Europe": {"DE", "FR", "GB", "IT", "ES", "PL", "NL", "SE", "AT", "CH", "BE"},
        "North America": {"US", "CA", "MX"},
        "South America": {"BR", "AR", "CO", "CL", "PE"},
        "Asia": {"CN", "IN", "JP", "KR", "IR"},
        "Africa": {"ZA", "EG", "NG"},
        "Oceania": {"AU", "NZ"}
    }

    # Falls "year" als String gespeichert ist, in numerischen Wert umwandeln
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

    # Regionen durchgehen & Diagramme erstellen
    fig, axes = plt.subplots(2, 3, figsize=(15, 8), sharex=True, sharey=True)  # Einheitliche Achsen für bessere Vergleichbarkeit
    axes = axes.flatten()  # 2D-Array in 1D-Array umwandeln

    # Speichert alle Linien für die gemeinsame Legende
    handles, labels = [], []

    for i, (region, countries) in enumerate(regions.items()):
        df_region = df[df["Country"].isin(countries)]
        yearly_trends_region = df_region.groupby("Year")[pollutants].mean()
        yearly_trends_region = yearly_trends_region.loc[(yearly_trends_region.index > 2014) & (yearly_trends_region.index < 2025)]
        
        ax = axes[i]
        for pollutant in pollutants:
            line, = ax.plot(yearly_trends_region.index, yearly_trends_region[pollutant], marker='o', linestyle='-', label=pollutant)
            
            # Speichert eine Linie pro Schadstoff für die gemeinsame Legende
            if i == 0:  
                handles.append(line)
                labels.append(pollutant)

        ax.set_title(region)
        ax.set_xlabel("Jahr")
        ax.set_ylabel("Mittelwert")
        ax.grid(True)

    for ax in axes[:3]:  # Die ersten drei Subplots sind in der oberen Reihe
        ax.xaxis.set_tick_params(labelbottom=True)

    # Gemeinsame Legende unterhalb der Subplots platzieren
    fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(pollutants))

    # Layout optimieren
    plt.tight_layout();

In [ ]:
# Sechs Top-Länder in Europa

import matplotlib.pyplot as plt

# Schadstoffe, die analysiert werden sollen
pollutants = ["co", "no2", "o3", "so2", "pm10", "pm25"]

# Europäische Länder definieren
european_countries = {"DE", "FR", "GB", "IT", "ES", "PL", "NL", "SE", "AT", "CH", "BE"}

# Falls "year" als String gespeichert ist, in numerischen Wert umwandeln
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Nur europäische Länder auswählen
df_europe = df[df["Country"].isin(european_countries)]

# Länder mit den meisten Messwerten identifizieren
top_countries = df_europe["Country"].value_counts().nlargest(6).index  # Falls nur 6 Länder visualisiert werden sollen

# Subplots für die gewählten Länder erstellen
fig, axes = plt.subplots(2, 3, figsize=(15, 8), sharex=True, sharey=True)  # 2 Reihen, 3 Spalten
axes = axes.flatten()

for i, country in enumerate(top_countries):
    df_country = df_europe[df_europe["Country"] == country]
    yearly_trends_country = df_country.groupby("year")[pollutants].mean()
    yearly_trends_country = yearly_trends_country.loc[(yearly_trends_country.index > 2014) & (yearly_trends_country.index < 2025)]
    
    ax = axes[i]
    for pollutant in pollutants:
        ax.plot(yearly_trends_country.index, yearly_trends_country[pollutant], marker='o', linestyle='-', label=pollutant)
    
    ax.set_title(country)
    ax.set_xlabel("Jahr")
    ax.set_ylabel("Mittelwert")
    ax.grid(True)

# Gemeinsame Legende unterhalb der Subplots platzieren
fig.legend(pollutants, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(pollutants))

# Layout optimieren
plt.tight_layout();

In [ ]:
# Korrekation zwischen Ozon und anderen Faktoren

# Relevante Spalten auswählen
pollutants = ["co", "no2", "so2", "pm10", "pm25", "o3"]
weather_vars = ["temperature", "pressure", "humidity", "dew", "wind-speed", "wind-gust"]

# DataFrame mit nur den relevanten Spalten (fehlende Werte entfernen)
df_ozone_corr = df[pollutants + weather_vars].dropna()

# Korrelationsmatrix berechnen
corr_matrix_ozone = df_ozone_corr.corr()

# Heatmap der Korrelationen zwischen Ozon und anderen Faktoren
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix_ozone, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Korrelation zwischen Ozon (O₃) und anderen Faktoren");


In [ ]:
# Durchschnittliche Ozonwerte pro Monat berechnen
monthly_o3 = df.groupby("month")["o3"].mean()

# Liniendiagramm für die saisonale Entwicklung von Ozon erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_o3.index, monthly_o3.values, marker='o', linestyle='-', color='b')
plt.xlabel("Monat")
plt.ylabel("Mittlere O₃-Konzentration")
plt.title("Saisonale Entwicklung der Ozonwerte (O₃)")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.grid(True);

In [ ]:
# Länder nach Hemisphäre aufteilen
northern_hemisphere_countries = {
    "US", "CA", "MX", "DE", "FR", "GB", "IT", "ES", "PL", "NL", "SE", "AT", "CH", "BE", "RU", "CN", "JP", "IN", "KR"
}
southern_hemisphere_countries = {
    "AU", "NZ", "AR", "BR", "ZA", "CL", "ID", "PE", "BO", "EC", "PY", "UY", "MG"
}

# Daten für Nord- und Südhalbkugel filtern
df_north = df[df["Country"].isin(northern_hemisphere_countries)]
df_south = df[df["Country"].isin(southern_hemisphere_countries)]

# Durchschnittliche Ozonwerte pro Monat für beide Hemisphären berechnen
monthly_o3_north = df_north.groupby("month")["o3"].mean()
monthly_o3_south = df_south.groupby("month")["o3"].mean()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_o3_north.index, monthly_o3_north.values, marker='o', linestyle='-', label="Nordhalbkugel", color='b')
plt.plot(monthly_o3_south.index, monthly_o3_south.values, marker='o', linestyle='-', label="Südhalbkugel", color='r')

plt.xlabel("Monat")
plt.ylabel("Mittlere O₃-Konzentration")
plt.title("Vergleich der Ozonwerte (O₃) zwischen Nord- und Südhalbkugel")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.legend()
plt.grid(True);



In [ ]:
import matplotlib.pyplot as plt

# Europäische Länder definieren
european_countries = {"DE", "FR", "GB", "IT", "ES", "PL", "NL", "SE", "AT", "CH", "BE"}

# Falls "year" als String gespeichert ist, in numerischen Wert umwandeln
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Nur europäische Länder auswählen
df_europe = df[df["Country"].isin(european_countries)]

# Jahre 2014 & 2025 aus der Analyse entfernen
df_europe = df_europe[(df_europe["year"] > 2014) & (df_europe["year"] < 2025)]

# Nur Länder behalten, die tatsächlich O₃-Werte haben
countries_with_o3 = df_europe.groupby("Country")["o3"].count()
valid_countries = countries_with_o3[countries_with_o3 > 0].index  # Länder mit vorhandenen O₃-Werten

# DataFrame auf diese Länder filtern
df_europe = df_europe[df_europe["Country"].isin(valid_countries)]

# Standardabweichung (Schwankungsstärke) von O₃ pro Jahr & Land berechnen
ozone_volatility = df_europe.groupby(["year", "Country"])["o3"].std().unstack()

# Diagramm erstellen
fig, ax = plt.subplots(figsize=(12, 6))
handles = []  # Zum Speichern der Linien für die Legende
labels = []   # Zum Speichern der Ländernamen

for country in ozone_volatility.columns:
    line, = ax.plot(ozone_volatility.index, ozone_volatility[country], marker='o', linestyle='-', label=country)
    handles.append(line)
    labels.append(country)

ax.set_xlabel("Jahr")
ax.set_ylabel("Standardabweichung von O₃ (Schwankungsstärke)")
ax.set_title("Jährliche Schwankungsstärke der Ozonwerte in europäischen Ländern (nur Länder mit Daten)")
ax.grid(True)

# Legende unterhalb des Plots platzieren
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.15), ncol=len(valid_countries))

# Layout optimieren
plt.tight_layout();